# Airfoil's Aerodynamic Coefficient Prediction using ANNs

In [1]:
import os
os.environ['TF_ENABLE_ONEDNN_OPTS'] = '0'
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '1'

In [2]:
# importing the dependencies
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error, r2_score
from tensorflow import keras
from keras import Input
from keras.models import Sequential
from keras.layers import Dense
from keras.optimizers import Adam
from keras.metrics import RootMeanSquaredError
from keras.callbacks import ReduceLROnPlateau
import matplotlib.pyplot as plt

In [3]:
# reading the csv file
df4 = pd.read_csv("/home/nevilcp/ML_Aero/results/NACA4D_10/NACA4D_10.csv")
df5 = pd.read_csv("/home/nevilcp/ML_Aero/results/NACA5D_10/NACA5D_10.csv")

In [4]:
# printing the first 10 rows of the dataset of NACA4D_10
df4.head(10)

,t,m,p,yU1,yU2,yU3,yU4,yU5,yU6,yU7,...,yL7,yL8,yL9,yL10,alpha,M,Re,CL,CD,Cm
0,5,0,5,0.009825,0.017881,0.023099,0.02499,0.023669,0.019965,0.014949,...,-0.014949,-0.009639,-0.004948,-0.001694,-10.0,0.2,100000,-1.158,0.02657,0.005
1,5,0,5,0.009825,0.017881,0.023099,0.02499,0.023669,0.019965,0.014949,...,-0.014949,-0.009639,-0.004948,-0.001694,-9.0,0.2,100000,-1.043,0.02194,0.004
2,5,0,5,0.009825,0.017881,0.023099,0.02499,0.023669,0.019965,0.014949,...,-0.014949,-0.009639,-0.004948,-0.001694,-8.0,0.2,100000,-0.928,0.01876,0.004
3,5,0,5,0.009825,0.017881,0.023099,0.02499,0.023669,0.019965,0.014949,...,-0.014949,-0.009639,-0.004948,-0.001694,-7.0,0.2,100000,-0.813,0.01463,0.004
4,5,0,5,0.009825,0.017881,0.023099,0.02499,0.023669,0.019965,0.014949,...,-0.014949,-0.009639,-0.004948,-0.001694,-6.0,0.2,100000,-0.697,0.02806,0.003
5,5,0,5,0.009825,0.017881,0.023099,0.02499,0.023669,0.019965,0.014949,...,-0.014949,-0.009639,-0.004948,-0.001694,-5.0,0.2,100000,-0.581,0.01660,0.003
6,5,0,5,0.009825,0.017881,0.023099,0.02499,0.023669,0.019965,0.014949,...,-0.014949,-0.009639,-0.004948,-0.001694,-4.0,0.2,100000,-0.465,0.01146,0.002
7,5,0,5,0.009825,0.017881,0.023099,0.02499,0.023669,0.019965,0.014949,...,-0.014949,-0.009639,-0.004948,-0.001694,-3.0,0.2,100000,-0.349,0.00893,0.002
8,5,0,5,0.009825,0.017881,0.023099,0.02499,0.023669,0.019965,0.014949,...,-0.014949,-0.009639,-0.004948,-0.001694,-2.0,0.2,100000,-0.233,0.01393,0.001
9,5,0,5,0.009825,0.017881,0.023099,0.02499,0.023669,0.019965,0.014949,...,-0.014949,-0.009639,-0.004948,-0.001694,-1.0,0.2,100000,-0.116,0.01358,0.001


In [5]:
# printing the first 10 rows of the dataset of NACA5D_10
df5.head(10)

,t,CL_design,p,s,yU1,yU2,yU3,yU4,yU5,yU6,...,yL7,yL8,yL9,yL10,alpha,M,Re,CL,CD,Cm
0,5,0.0,5,0,0.009825,0.017881,0.023099,0.02499,0.023669,0.019965,...,-0.014949,-0.009639,-0.004948,-0.001694,-10.0,0.1,100000,-1.140,0.02494,0.005
1,5,0.0,5,0,0.009825,0.017881,0.023099,0.02499,0.023669,0.019965,...,-0.014949,-0.009639,-0.004948,-0.001694,-9.0,0.1,100000,-1.027,0.02116,0.004
2,5,0.0,5,0,0.009825,0.017881,0.023099,0.02499,0.023669,0.019965,...,-0.014949,-0.009639,-0.004948,-0.001694,-8.0,0.1,100000,-0.914,0.01697,0.004
3,5,0.0,5,0,0.009825,0.017881,0.023099,0.02499,0.023669,0.019965,...,-0.014949,-0.009639,-0.004948,-0.001694,-7.0,0.1,100000,-0.800,0.01364,0.003
4,5,0.0,5,0,0.009825,0.017881,0.023099,0.02499,0.023669,0.019965,...,-0.014949,-0.009639,-0.004948,-0.001694,-6.0,0.1,100000,-0.687,0.02483,0.003
5,5,0.0,5,0,0.009825,0.017881,0.023099,0.02499,0.023669,0.019965,...,-0.014949,-0.009639,-0.004948,-0.001694,-5.0,0.1,100000,-0.572,0.01565,0.002
6,5,0.0,5,0,0.009825,0.017881,0.023099,0.02499,0.023669,0.019965,...,-0.014949,-0.009639,-0.004948,-0.001694,-4.0,0.1,100000,-0.458,0.01114,0.002
7,5,0.0,5,0,0.009825,0.017881,0.023099,0.02499,0.023669,0.019965,...,-0.014949,-0.009639,-0.004948,-0.001694,-3.0,0.1,100000,-0.344,0.00882,0.001
8,5,0.0,5,0,0.009825,0.017881,0.023099,0.02499,0.023669,0.019965,...,-0.014949,-0.009639,-0.004948,-0.001694,-2.0,0.1,100000,-0.229,0.01391,0.001
9,5,0.0,5,0,0.009825,0.017881,0.023099,0.02499,0.023669,0.019965,...,-0.014949,-0.009639,-0.004948,-0.001694,-1.0,0.1,100000,-0.115,0.01357,0.000


In [6]:
# checking for missing values in the NACA4D_10 dataset
df4.isnull().sum()

t        0
m        0
p        0
yU1      0
yU2      0
yU3      0
yU4      0
yU5      0
yU6      0
yU7      0
yU8      0
yU9      0
yU10     0
yL1      0
yL2      0
yL3      0
yL4      0
yL5      0
yL6      0
yL7      0
yL8      0
yL9      0
yL10     0
alpha    0
M        0
Re       0
CL       0
CD       0
Cm       0
dtype: int64

In [7]:
# checking for missing values in the NACA5D_10 dataset
df5.isnull().sum()

t            0
CL_design    0
p            0
s            0
yU1          0
yU2          0
yU3          0
yU4          0
yU5          0
yU6          0
yU7          0
yU8          0
yU9          0
yU10         0
yL1          0
yL2          0
yL3          0
yL4          0
yL5          0
yL6          0
yL7          0
yL8          0
yL9          0
yL10         0
alpha        0
M            0
Re           0
CL           0
CD           0
Cm           0
dtype: int64

In [8]:
# description of the NACA4D_10 dataset
df4.describe()

,t,m,p,yU1,yU2,yU3,yU4,yU5,yU6,yU7,...,yL7,yL8,yL9,yL10,alpha,M,Re,CL,CD,Cm
count,176400.000000,176400.000000,176400.000000,176400.000000,176400.000000,176400.000000,176400.000000,176400.000000,176400.000000,176400.000000,...,176400.000000,176400.000000,176400.000000,176400.000000,176400.000000,176400.00000,176400.000000,176400.000000,176400.000000,176400.000000
mean,20.000000,4.500000,40.000000,0.056883,0.096159,0.125194,0.138691,0.136083,0.120107,0.094865,...,-0.025367,-0.013472,-0.006526,-0.003090,0.000000,0.20000,300000.000000,0.692283,0.026646,-0.134151
std,10.000028,2.872289,22.912943,0.030652,0.043042,0.052297,0.056240,0.054535,0.048225,0.039002,...,0.037872,0.026573,0.014169,0.004346,6.055318,0.08165,141421.757093,0.926925,0.010960,0.113007
min,5.000000,0.000000,5.000000,0.009825,0.017881,0.023099,0.024990,0.023669,0.019965,0.014949,...,-0.104640,-0.067474,-0.034638,-0.011856,-10.000000,0.10000,100000.000000,-1.489000,0.001800,-0.514000
25%,10.000000,2.000000,22.500000,0.031423,0.060652,0.084089,0.094866,0.094677,0.083039,0.065317,...,-0.053389,-0.032844,-0.016811,-0.006339,-5.000000,0.10000,200000.000000,-0.011000,0.018990,-0.199000
50%,20.000000,4.500000,40.000000,0.054270,0.096111,0.124907,0.138240,0.136379,0.119788,0.094447,...,-0.025009,-0.014037,-0.006982,-0.003241,-0.000000,0.20000,300000.000000,0.691000,0.025390,-0.106000
75%,30.000000,7.000000,57.500000,0.078524,0.129458,0.166614,0.182079,0.176607,0.156303,0.123175,...,0.001474,0.004181,0.002635,-0.000235,5.000000,0.30000,400000.000000,1.379000,0.032540,-0.046000
max,35.000000,9.000000,75.000000,0.149810,0.208025,0.246146,0.264431,0.256034,0.230429,0.194578,...,0.074748,0.071348,0.042540,0.011816,10.000000,0.30000,500000.000000,3.599000,0.235470,0.025000


In [9]:
# description of the NACA5D_10 dataset
df5.describe()

,t,CL_design,p,s,yU1,yU2,yU3,yU4,yU5,yU6,...,yL7,yL8,yL9,yL10,alpha,M,Re,CL,CD,Cm
count,236670.000000,236670.000000,236670.000000,236670.000000,236670.000000,236670.000000,236670.000000,236670.000000,236670.000000,236670.000000,...,236670.000000,236670.000000,236670.000000,236670.000000,236670.000000,236670.000000,236670.000000,236670.000000,236670.000000,236670.000000
mean,19.964508,0.841171,27.648625,0.493345,0.061993,0.102135,0.127800,0.134156,0.122524,0.098694,...,-0.050760,-0.035252,-0.019158,-0.006666,0.000000,0.200000,300000.000000,0.219104,0.025913,0.008665
std,10.002176,0.595675,16.859596,0.499957,0.035497,0.050154,0.062040,0.067278,0.064190,0.054114,...,0.038246,0.024719,0.012716,0.004107,6.055314,0.081686,141421.655012,0.926011,0.011283,0.092562
min,5.000000,0.000000,5.000000,0.000000,0.002846,-0.006454,-0.022993,-0.039093,-0.048037,-0.046811,...,-0.171128,-0.127512,-0.072044,-0.023250,-10.000000,0.100000,100000.000000,-2.607000,0.002690,-0.169000
25%,10.000000,0.200000,15.000000,0.000000,0.033635,0.064809,0.081379,0.082846,0.074521,0.059894,...,-0.078123,-0.052613,-0.027989,-0.009654,-5.000000,0.100000,200000.000000,-0.485000,0.018260,-0.031000
50%,20.000000,0.800000,25.000000,0.000000,0.057851,0.101474,0.127721,0.133115,0.120174,0.099175,...,-0.050658,-0.034769,-0.018871,-0.006613,-0.000000,0.200000,300000.000000,0.218000,0.024650,-0.004000
75%,30.000000,1.400000,45.000000,1.000000,0.085249,0.134294,0.169672,0.179965,0.165684,0.137987,...,-0.022239,-0.016698,-0.009816,-0.003398,5.000000,0.300000,400000.000000,0.928000,0.031580,0.018000
max,35.000000,1.800000,75.000000,1.000000,0.165549,0.250220,0.311403,0.334353,0.308679,0.262679,...,0.048283,0.022775,0.009953,0.002097,10.000000,0.300000,500000.000000,3.020000,0.617020,0.453000


### Dropping columns

In [10]:
# dropping columns t, m & p in the NACA4D_10 dataset
df4 = df4.drop(columns=['t', 'm', 'p'], axis=1)
df4.head(10)

,yU1,yU2,yU3,yU4,yU5,yU6,yU7,yU8,yU9,yU10,...,yL7,yL8,yL9,yL10,alpha,M,Re,CL,CD,Cm
0,0.009825,0.017881,0.023099,0.02499,0.023669,0.019965,0.014949,0.009639,0.004948,0.001694,...,-0.014949,-0.009639,-0.004948,-0.001694,-10.0,0.2,100000,-1.158,0.02657,0.005
1,0.009825,0.017881,0.023099,0.02499,0.023669,0.019965,0.014949,0.009639,0.004948,0.001694,...,-0.014949,-0.009639,-0.004948,-0.001694,-9.0,0.2,100000,-1.043,0.02194,0.004
2,0.009825,0.017881,0.023099,0.02499,0.023669,0.019965,0.014949,0.009639,0.004948,0.001694,...,-0.014949,-0.009639,-0.004948,-0.001694,-8.0,0.2,100000,-0.928,0.01876,0.004
3,0.009825,0.017881,0.023099,0.02499,0.023669,0.019965,0.014949,0.009639,0.004948,0.001694,...,-0.014949,-0.009639,-0.004948,-0.001694,-7.0,0.2,100000,-0.813,0.01463,0.004
4,0.009825,0.017881,0.023099,0.02499,0.023669,0.019965,0.014949,0.009639,0.004948,0.001694,...,-0.014949,-0.009639,-0.004948,-0.001694,-6.0,0.2,100000,-0.697,0.02806,0.003
5,0.009825,0.017881,0.023099,0.02499,0.023669,0.019965,0.014949,0.009639,0.004948,0.001694,...,-0.014949,-0.009639,-0.004948,-0.001694,-5.0,0.2,100000,-0.581,0.01660,0.003
6,0.009825,0.017881,0.023099,0.02499,0.023669,0.019965,0.014949,0.009639,0.004948,0.001694,...,-0.014949,-0.009639,-0.004948,-0.001694,-4.0,0.2,100000,-0.465,0.01146,0.002
7,0.009825,0.017881,0.023099,0.02499,0.023669,0.019965,0.014949,0.009639,0.004948,0.001694,...,-0.014949,-0.009639,-0.004948,-0.001694,-3.0,0.2,100000,-0.349,0.00893,0.002
8,0.009825,0.017881,0.023099,0.02499,0.023669,0.019965,0.014949,0.009639,0.004948,0.001694,...,-0.014949,-0.009639,-0.004948,-0.001694,-2.0,0.2,100000,-0.233,0.01393,0.001
9,0.009825,0.017881,0.023099,0.02499,0.023669,0.019965,0.014949,0.009639,0.004948,0.001694,...,-0.014949,-0.009639,-0.004948,-0.001694,-1.0,0.2,100000,-0.116,0.01358,0.001


In [11]:
# dropping columns t, CL_design, p & s in the NACA5D_10 dataset
df5 = df5.drop(columns=['t', 'CL_design', 'p', 's'], axis=1)
df5.head(10)

,yU1,yU2,yU3,yU4,yU5,yU6,yU7,yU8,yU9,yU10,...,yL7,yL8,yL9,yL10,alpha,M,Re,CL,CD,Cm
0,0.009825,0.017881,0.023099,0.02499,0.023669,0.019965,0.014949,0.009639,0.004948,0.001694,...,-0.014949,-0.009639,-0.004948,-0.001694,-10.0,0.1,100000,-1.140,0.02494,0.005
1,0.009825,0.017881,0.023099,0.02499,0.023669,0.019965,0.014949,0.009639,0.004948,0.001694,...,-0.014949,-0.009639,-0.004948,-0.001694,-9.0,0.1,100000,-1.027,0.02116,0.004
2,0.009825,0.017881,0.023099,0.02499,0.023669,0.019965,0.014949,0.009639,0.004948,0.001694,...,-0.014949,-0.009639,-0.004948,-0.001694,-8.0,0.1,100000,-0.914,0.01697,0.004
3,0.009825,0.017881,0.023099,0.02499,0.023669,0.019965,0.014949,0.009639,0.004948,0.001694,...,-0.014949,-0.009639,-0.004948,-0.001694,-7.0,0.1,100000,-0.800,0.01364,0.003
4,0.009825,0.017881,0.023099,0.02499,0.023669,0.019965,0.014949,0.009639,0.004948,0.001694,...,-0.014949,-0.009639,-0.004948,-0.001694,-6.0,0.1,100000,-0.687,0.02483,0.003
5,0.009825,0.017881,0.023099,0.02499,0.023669,0.019965,0.014949,0.009639,0.004948,0.001694,...,-0.014949,-0.009639,-0.004948,-0.001694,-5.0,0.1,100000,-0.572,0.01565,0.002
6,0.009825,0.017881,0.023099,0.02499,0.023669,0.019965,0.014949,0.009639,0.004948,0.001694,...,-0.014949,-0.009639,-0.004948,-0.001694,-4.0,0.1,100000,-0.458,0.01114,0.002
7,0.009825,0.017881,0.023099,0.02499,0.023669,0.019965,0.014949,0.009639,0.004948,0.001694,...,-0.014949,-0.009639,-0.004948,-0.001694,-3.0,0.1,100000,-0.344,0.00882,0.001
8,0.009825,0.017881,0.023099,0.02499,0.023669,0.019965,0.014949,0.009639,0.004948,0.001694,...,-0.014949,-0.009639,-0.004948,-0.001694,-2.0,0.1,100000,-0.229,0.01391,0.001
9,0.009825,0.017881,0.023099,0.02499,0.023669,0.019965,0.014949,0.009639,0.004948,0.001694,...,-0.014949,-0.009639,-0.004948,-0.001694,-1.0,0.1,100000,-0.115,0.01357,0.000


In [12]:
# merging the two datasets
df = pd.concat([df4, df5], ignore_index=True)

In [13]:
# shuffling all the rows
df = df.sample(frac=1)

In [14]:
# defining the feature and target columns
X = df.drop(columns=['CL', 'CD', 'Cm']) 
y = df[['CL', 'CD', 'Cm']]

In [15]:
# displaying the features table
X.head(10)

,yU1,yU2,yU3,yU4,yU5,yU6,yU7,yU8,yU9,yU10,...,yL4,yL5,yL6,yL7,yL8,yL9,yL10,alpha,M,Re
302528,0.061675,0.110384,0.142699,0.154975,0.147623,0.125228,0.094242,0.061117,0.031734,0.011337,...,-0.144916,-0.136403,-0.114341,-0.085133,-0.054541,-0.027626,-0.008961,-8.0,0.2,200000
397606,0.156298,0.225753,0.264688,0.271862,0.250996,0.207796,0.153057,0.097036,0.048604,0.015476,...,-0.078769,-0.088171,-0.081149,-0.064314,-0.043338,-0.023296,-0.008732,3.0,0.3,400000
268587,0.045037,0.083315,0.110798,0.118433,0.107341,0.087401,0.063698,0.040110,0.020013,0.006403,...,-0.031609,-0.035055,-0.032757,-0.026276,-0.017903,-0.009751,-0.003760,8.0,0.2,500000
65636,0.069602,0.120493,0.156808,0.173500,0.170233,0.149833,0.117031,0.077888,0.040414,0.013149,...,-0.126663,-0.113935,-0.089732,-0.062419,-0.038051,-0.019140,-0.007102,1.0,0.2,100000
304212,0.082125,0.116529,0.144969,0.152947,0.142607,0.118907,0.088145,0.056322,0.028548,0.009454,...,-0.096738,-0.096184,-0.083437,-0.063625,-0.041640,-0.021703,-0.007655,-4.0,0.1,200000
236918,0.043673,0.066947,0.081885,0.086114,0.080178,0.066896,0.049666,0.031791,0.016175,0.005423,...,-0.063850,-0.062089,-0.053222,-0.040300,-0.026232,-0.013605,-0.004759,7.0,0.2,200000
61161,0.074574,0.126449,0.163355,0.179108,0.171812,0.147186,0.111865,0.072910,0.037374,0.012249,...,-0.120848,-0.112366,-0.092661,-0.067826,-0.043050,-0.022122,-0.007999,-1.0,0.2,300000
18118,0.028134,0.045646,0.055984,0.059332,0.055779,0.046973,0.035159,0.022626,0.011524,0.003817,...,-0.040630,-0.038929,-0.032929,-0.024672,-0.015956,-0.008280,-0.002958,6.0,0.3,300000
150369,0.030511,0.064433,0.097405,0.120407,0.127069,0.116260,0.092875,0.062284,0.031791,0.009297,...,0.019750,0.032359,0.036217,0.032579,0.022977,0.011444,0.002320,-1.0,0.3,100000
12828,0.068776,0.125166,0.161693,0.174931,0.165684,0.139753,0.104640,0.067474,0.034638,0.011856,...,-0.174931,-0.165684,-0.139753,-0.104640,-0.067474,-0.034638,-0.011856,8.0,0.1,100000


In [16]:
# displaying the targets table
y.head(10)

,CL,CD,Cm
302528,-1.052,0.03410,0.004
397606,1.607,0.03906,-0.030
268587,1.401,0.02484,-0.052
65636,0.688,0.03255,-0.118
304212,-0.231,0.01459,0.001
236918,0.997,0.02308,-0.008
61161,0.325,0.02721,-0.075
18118,0.858,0.01681,-0.016
150369,0.982,0.02179,-0.249
12828,1.144,0.04148,-0.020


## Splitting the Dataset

In [17]:
# splitting dataset into training, validation & testing dataset
X_train, X_temp, y_train, y_temp = train_test_split(X, y, train_size=0.7, random_state=42)
X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.5, random_state=42)

In [18]:
# normalizing the training, validation & testing datasets
X_scaler = StandardScaler()
X_train = X_scaler.fit_transform(X_train)
X_val = X_scaler.transform(X_val)
X_test = X_scaler.transform(X_test)

In [19]:
# displaying X_train, X_val & X_test after standardization
display(pd.DataFrame(X_train).head())
display(pd.DataFrame(X_val).head())
display(pd.DataFrame(X_test).head())

,0,1,2,3,4,5,6,7,8,9,...,13,14,15,16,17,18,19,20,21,22
0,1.399536,1.284345,1.179314,1.119781,1.075144,1.025661,0.957133,0.876663,0.834949,0.939331,...,-1.273664,-1.323254,-1.324817,-1.284296,-1.212827,-1.162329,-1.240785,-0.824292,-1.224660,0.706796
1,-0.635686,-0.366129,-0.342502,-0.398844,-0.472692,-0.519720,-0.519160,-0.472454,-0.387304,-0.190930,...,-2.008629,-2.071155,-2.090722,-2.023343,-1.879683,-1.735626,-1.646178,1.156681,-1.224660,-1.414318
2,1.412546,1.426044,1.427499,1.410615,1.417278,1.434285,1.417841,1.360630,1.314827,1.367350,...,-0.981584,-0.954245,-0.880998,-0.790541,-0.703573,-0.655495,-0.764107,-1.484617,-0.000673,-1.414318
3,0.918000,1.064628,1.140311,1.245587,1.390902,1.558492,1.694612,1.745538,1.756453,1.789708,...,-1.161685,-0.970259,-0.719676,-0.469300,-0.292072,-0.205622,-0.335196,0.661438,1.223314,1.413833
4,-0.611107,-0.594091,-0.592032,-0.576857,-0.556701,-0.532158,-0.502336,-0.465272,-0.421922,-0.365856,...,-0.578379,-0.553870,-0.527746,-0.495983,-0.455106,-0.408932,-0.357974,0.991600,-1.224660,1.413833


,0,1,2,3,4,5,6,7,8,9,...,13,14,15,16,17,18,19,20,21,22
0,-0.895909,-0.780881,-0.538004,-0.246713,0.074834,0.413912,0.691632,0.837605,0.859131,0.619088,...,1.342382,1.687366,2.014392,2.219263,2.247148,2.208555,2.074007,0.331276,1.223314,1.413833
1,-0.478837,-0.452686,-0.430856,-0.418149,-0.400315,-0.374931,-0.347372,-0.318323,-0.285908,-0.248914,...,-0.418319,-0.393826,-0.365431,-0.335642,-0.302701,-0.265936,-0.230699,1.486843,-0.000673,-0.707280
2,-0.903358,-0.971844,-0.989826,-0.974812,-0.947509,-0.911138,-0.862520,-0.800909,-0.742809,-0.718903,...,-0.177023,-0.153930,-0.136399,-0.123155,-0.107051,-0.073155,0.013038,-0.989373,-0.000673,1.413833
3,0.800030,0.923688,1.046533,1.203923,1.376566,1.523227,1.614968,1.628727,1.604529,1.560581,...,-0.408945,-0.184037,0.013344,0.168359,0.254726,0.278523,0.126917,-1.319536,-1.224660,-0.707280
4,-0.796474,-0.829930,-0.887221,-0.930674,-0.956055,-0.952991,-0.913228,-0.839435,-0.752622,-0.646682,...,-0.936530,-0.962627,-0.963587,-0.924118,-0.846840,-0.757311,-0.646976,-1.649698,-0.000673,-0.707280


,0,1,2,3,4,5,6,7,8,9,...,13,14,15,16,17,18,19,20,21,22
0,-0.143387,0.239372,0.158000,0.021841,-0.006153,0.011636,0.030444,0.029794,0.000571,-0.167572,...,1.629688,1.603726,1.587405,1.530021,1.428075,1.351207,1.310124,0.331276,1.223314,1.413833
1,0.288402,0.444616,0.529166,0.623186,0.755049,0.930131,1.172459,1.517850,1.752365,1.823352,...,-0.980971,-0.816530,-0.582376,-0.247375,0.207399,0.459116,0.378056,-0.163968,-0.000673,-0.000242
2,1.387158,1.494074,1.612683,1.739850,1.820245,1.783468,1.619602,1.427739,1.283788,1.218348,...,0.146294,0.244149,0.202512,0.094280,0.006749,-0.061059,-0.218066,-0.494130,1.223314,1.413833
3,1.243228,0.905220,1.034031,1.060343,1.046834,1.010925,0.951066,0.875536,0.836189,0.942093,...,-1.321120,-1.388665,-1.396405,-1.353393,-1.277750,-1.222171,-1.294563,-0.989373,1.223314,-0.707280
4,1.367326,1.440337,1.496976,1.488013,1.502713,1.538744,1.541414,1.488952,1.428279,1.386429,...,-0.098293,-0.075665,-0.008090,0.053251,0.085009,0.084744,-0.055903,-0.989373,1.223314,-0.707280


In [20]:
# displaying y_train, y_val & y_test after standardization
display(pd.DataFrame(y_train).head())
display(pd.DataFrame(y_val).head())
display(pd.DataFrame(y_test).head())

,CL,CD,Cm
265508,-0.343,0.02957,0.006
366257,0.370,0.03226,0.087
94501,-0.547,0.03781,-0.076
101423,1.568,0.03875,-0.207
186265,0.773,0.02164,-0.009


,CL,CD,Cm
170406,1.616,0.02315,-0.335
23140,1.305,0.03008,-0.034
186043,-0.756,0.01874,0.007
133898,0.025,0.02980,-0.201
299586,-1.614,0.03442,0.074


,CL,CD,Cm
161586,1.271,0.01860,-0.148
87621,0.819,0.02911,-0.223
406336,0.780,0.02923,-0.154
304945,-0.547,0.01706,0.007
147340,0.318,0.02982,-0.154


In [21]:
# defining the learning rate reduction callback
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.9, patience=5, min_lr=0.00001)

### Model [512, 256, 128, 3]

In [22]:
# RMSE and R² metric scores on test data for Model M_512_256_128
M_512_256_128_metrics = []

for i in range(20):
    print(f"\nRun {i+1}/20 for Model M_512_256_128...")

    M_512_256_128 = Sequential([
        Input(shape=(X_train.shape[1],)),
        Dense(512, activation='relu'),
        Dense(256, activation='relu'),
        Dense(128, activation='relu'),
        Dense(y_train.shape[1], activation='linear')
    ])

    M_512_256_128.compile(
        optimizer=Adam(learning_rate=5e-4, beta_1=0.9, beta_2=0.999),
        loss='mse',
        metrics=[RootMeanSquaredError()]
    )

    M_512_256_128.fit(
        X_train, y_train,
        epochs=50, batch_size=128,
        validation_data=(X_val, y_val),
        callbacks=[reduce_lr],
        verbose=0
    )

    y_pred = M_512_256_128.predict(X_test, verbose=0)
    r2 = r2_score(y_test, y_pred, multioutput='raw_values')
    rmse = np.sqrt(mean_squared_error(y_test, y_pred, multioutput='raw_values'))

    M_512_256_128_metrics.append({
        'CL_rmse': rmse[0], 'CD_rmse': rmse[1], 'Cm_rmse': rmse[2],
        'CL_r2': r2[0], 'CD_r2': r2[1], 'Cm_r2': r2[2]
    })

results_M_512_256_128_df = pd.DataFrame(M_512_256_128_metrics)

print(results_M_512_256_128_df.mean())



Run 1/20 for Model M_512_256_128...


I0000 00:00:1764219395.634620  131579 gpu_device.cc:2020] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 6155 MB memory:  -> device: 0, name: NVIDIA GeForce RTX 4060 Laptop GPU, pci bus id: 0000:01:00.0, compute capability: 8.9
I0000 00:00:1764219399.241366  131701 device_compiler.h:196] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.



Run 2/20 for Model M_512_256_128...

Run 3/20 for Model M_512_256_128...

Run 4/20 for Model M_512_256_128...

Run 5/20 for Model M_512_256_128...

Run 6/20 for Model M_512_256_128...

Run 7/20 for Model M_512_256_128...

Run 8/20 for Model M_512_256_128...

Run 9/20 for Model M_512_256_128...

Run 10/20 for Model M_512_256_128...

Run 11/20 for Model M_512_256_128...

Run 12/20 for Model M_512_256_128...

Run 13/20 for Model M_512_256_128...

Run 14/20 for Model M_512_256_128...

Run 15/20 for Model M_512_256_128...

Run 16/20 for Model M_512_256_128...

Run 17/20 for Model M_512_256_128...

Run 18/20 for Model M_512_256_128...

Run 19/20 for Model M_512_256_128...

Run 20/20 for Model M_512_256_128...
CL_rmse    0.002138
CD_rmse    0.003652
Cm_rmse    0.000947
CL_r2      0.999995
CD_r2      0.890248
Cm_r2      0.999941
dtype: float32
